# Exploratory Data Analysis: LUNA16 Lung Nodule Detection

This notebook performs an initial exploration of the LUNA16 dataset, focusing on CT scan visualization, nodule distribution, and patch sampling. It assumes access to the dataset files (annotations.csv, candidates.csv, and .mhd scans) mounted via Google Drive or locally.

## Setup
- Mount Google Drive if using Colab.
- Install required libraries.

In [ ]:
# Install dependencies (run once)
!pip install SimpleITK pandas numpy matplotlib seaborn scikit-learn opencv-python

# Imports
import SimpleITK as sitk
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob
import os
from sklearn.model_selection import train_test_split
import cv2

# Mount Drive (Colab)
from google.colab import drive
drive.mount('/content/drive')

# Paths
DATA_PATH = '/content/drive/MyDrive/LUNA16/'
ANNOTATIONS_FILE = os.path.join(DATA_PATH, 'annotations.csv')
CANDIDATES_FILE = os.path.join(DATA_PATH, 'candidates.csv')

print('Setup complete.')

## 1. Load and Inspect Metadata

Examine the annotations (nodules) and candidates (potential nodules) CSV files.

In [ ]:
# Load CSVs
annotations = pd.read_csv(ANNOTATIONS_FILE)
candidates = pd.read_csv(CANDIDATES_FILE)

print('Annotations (Nodules):')
print(annotations.head())
print(f'Shape: {annotations.shape}')

print('\nCandidates:')
print(candidates.head())
print(f'Shape: {candidates.shape}')

# Class distribution in candidates
class_dist = candidates['class'].value_counts().sort_index()
print(f'\nClass Distribution:\n{class_dist}')

# Plot distribution
plt.figure(figsize=(8, 4))
sns.barplot(x=class_dist.index, y=class_dist.values)
plt.title('Distribution of Candidates (0: Non-Nodule, 1: Nodule)')
plt.xlabel('Class')
plt.ylabel('Count')
plt.show()

## 2. Load a Sample CT Scan

Load and visualize a single .mhd file to understand the 3D structure.

In [ ]:
def load_sample_scan(seriesuid):
    mhd_files = glob(os.path.join(DATA_PATH, 'subset*/', f'{seriesuid}.mhd'))
    if not mhd_files:
        raise ValueError(f'No .mhd file found for {seriesuid}')
    
    itkimage = sitk.ReadImage(mhd_files[0])
    ct_scan = sitk.GetArrayFromImage(itkimage)  # Shape: (slices, height, width)
    
    # Convert to Hounsfield (simple shift for visualization)
    ct_scan = ct_scan.astype(np.float32) - 1024
    
    return ct_scan

# Sample seriesuid (from annotations)
sample_seriesuid = annotations['seriesuid'].iloc[0]
print(f'Loading sample scan: {sample_seriesuid}')
ct_scan = load_sample_scan(sample_seriesuid)
print(f'Scan shape: {ct_scan.shape}')
print(f'Value range: [{ct_scan.min():.1f}, {ct_scan.max():.1f}] HU')

In [ ]:
# Visualize middle slices
fig, axes = plt.subplots(1, 10, figsize=(20, 4))
mid_slice = ct_scan.shape[0] // 2
start = max(0, mid_slice - 4)
end = min(ct_scan.shape[0], mid_slice + 5)

for i, z in enumerate(range(start, end)):
    axes[i].imshow(ct_scan[z], cmap='gray', vmin=-1000, vmax=400)
    axes[i].set_title(f'Slice {z}')
    axes[i].axis('off')

plt.suptitle('Sample CT Slices (Axial View)')
plt.tight_layout()
plt.show()

## 3. Visualize Nodule Locations

Overlay nodule coordinates on a sample slice for context.

In [ ]:
def world_to_voxel_coords(coords, origin, spacing):
    voxel_coords = np.round((coords - origin) / spacing).astype(int)
    return voxel_coords

# Load metadata for sample scan
itkimage = sitk.ReadImage(glob(os.path.join(DATA_PATH, 'subset*/', f'{sample_seriesuid}.mhd'))[0])
origin = np.array(itkimage.GetOrigin())
spacing = np.array(itkimage.GetSpacing())

# Get nodules for this scan
sample_nodules = annotations[annotations['seriesuid'] == sample_seriesuid]
print(f'Nodules in sample scan: {len(sample_nodules)}')

# Plot middle slice with nodule markers
mid_slice = ct_scan.shape[0] // 2
slice_img = ct_scan[mid_slice]

fig, ax = plt.subplots(1, 1, figsize=(8, 8))
ax.imshow(slice_img, cmap='gray', vmin=-1000, vmax=400)

# Mark nodule positions (project to this slice if nearby)
for _, nodule in sample_nodules.iterrows():
    coords = np.array([nodule['coordX'], nodule['coordY'], nodule['coordZ']])
    voxel = world_to_voxel_coords(coords, origin, spacing)
    if abs(voxel[0] - mid_slice) < 5:  # Nearby slices
        ax.plot(voxel[1], voxel[2], 'r+', markersize=10, markeredgewidth=2)

ax.set_title(f'Middle Slice with Nodule Markers (Red +)')
ax.axis('off')
plt.show()

## 4. Sample Patch Extraction

Extract and visualize sample patches around a nodule and a non-nodule.

In [ ]:
def extract_patch(ct_scan, coords, origin, spacing, patch_size=64):
    voxel = world_to_voxel_coords(coords, origin, spacing)
    z, y, x = voxel
    z = np.clip(z, 0, ct_scan.shape[0] - 1)
    half = patch_size // 2
    y_start = np.clip(y - half, 0, ct_scan.shape[1])
    y_end = np.clip(y + half, 0, ct_scan.shape[1])
    x_start = np.clip(x - half, 0, ct_scan.shape[2])
    x_end = np.clip(x + half, 0, ct_scan.shape[2])
    
    patch = ct_scan[z, y_start:y_end, x_start:x_end]
    # Resize if needed
    if patch.shape != (patch_size, patch_size):
        patch = cv2.resize(patch, (patch_size, patch_size), interpolation=cv2.INTER_LINEAR)
    return patch

# Sample nodule patch
if len(sample_nodules) > 0:
    nodule_row = sample_nodules.iloc[0]
    nodule_coords = np.array([nodule_row['coordX'], nodule_row['coordY'], nodule_row['coordZ']])
    nodule_patch = extract_patch(ct_scan, nodule_coords, origin, spacing)

# Sample non-nodule from candidates
non_nodule_candidates = candidates[(candidates['seriesuid'] == sample_seriesuid) & (candidates['class'] == 0)]
if len(non_nodule_candidates) > 0:
    non_nodule_row = non_nodule_candidates.iloc[0]
    non_nodule_coords = np.array([non_nodule_row['coordX'], non_nodule_row['coordY'], non_nodule_row['coordZ']])
    non_nodule_patch = extract_patch(ct_scan, non_nodule_coords, origin, spacing)

# Visualize patches
fig, axes = plt.subplots(1, 2, figsize=(10, 5))
if 'nodule_patch' in locals():
    axes[0].imshow(nodule_patch, cmap='gray', vmin=-1000, vmax=400)
    axes[0].set_title('Nodule Patch')
    axes[0].axis('off')
if 'non_nodule_patch' in locals():
    axes[1].imshow(non_nodule_patch, cmap='gray', vmin=-1000, vmax=400)
    axes[1].set_title('Non-Nodule Patch')
    axes[1].axis('off')
plt.tight_layout()
plt.show()

## 5. Dataset Statistics

Summary stats across scans.

In [ ]:
# Scans per subset
subsets = glob(os.path.join(DATA_PATH, 'subset*'))
scan_counts = {os.path.basename(s): len(glob(os.path.join(s, '*.mhd'))) for s in subsets}
print('Scans per subset:')
for k, v in scan_counts.items():
    print(f'{k}: {v}')

# Nodules per scan
nodules_per_scan = annotations['seriesuid'].value_counts()
print(f'\nNodules per scan - Min: {nodules_per_scan.min()}, Max: {nodules_per_scan.max()}, Mean: {nodules_per_scan.mean():.1f}')

# Plot nodules distribution
plt.figure(figsize=(8, 5))
nodules_per_scan.hist(bins=20)
plt.title('Distribution of Nodules per Scan')
plt.xlabel('Number of Nodules')
plt.ylabel('Frequency')
plt.show()

## 6. Balanced Sampling Preview

Simulate balanced sampling for training.

In [ ]:
positive_count = len(annotations)
negative_candidates = candidates[candidates['class'] == 0]
balanced_negatives = negative_candidates.sample(n=positive_count, random_state=42)

print(f'Original negatives: {len(negative_candidates)}')
print(f'Balanced negatives: {len(balanced_negatives)}')
print(f'Total balanced samples: {len(annotations) + len(balanced_negatives)}')

# Split preview
all_samples = pd.concat([annotations.assign(class=1)[['seriesuid', 'coordX', 'coordY', 'coordZ', 'class']], 
                         balanced_negatives[['seriesuid', 'coordX', 'coordY', 'coordZ', 'class']]], ignore_index=True)
train_samples, test_samples = train_test_split(all_samples, test_size=0.2, random_state=42, stratify=all_samples['class'])

print(f'\nTrain: {len(train_samples)} samples ({np.sum(train_samples["class"] == 1)} positives)')
print(f'Test: {len(test_samples)} samples ({np.sum(test_samples["class"] == 1)} positives)')

## Conclusion

This EDA reveals a highly imbalanced dataset (many more non-nodules), justifying balancing and augmentation. Scans show typical lung CT densities (-1000 to +400 HU). Nodules are sparse, emphasizing the need for precise patch extraction. Proceed to full preprocessing and modeling.